In [ ]:
function correlation_matrix(Lx, n_occupied)
    
    # create Hamiltonian matrix for hopping fermions
    t = 1.
    mu = 0.
    Hmatrix = SymTridiagonal(mu .* ones(Lx), t .* ones(Lx-1))

    # Make the correlation matrix (\Lambda) from its diagonal form (\Gamma)
    # \Gamma has n_occupied 1s--corresponding to the lowest eigenvalues--and the rest of the diagonal 0s
    # we have $\Lambda = U^{*} \Gamma U^{T}$
    vecs = eigfact(Hmatrix, 1:n_occupied)[:vectors]
    return conj(vecs) * transpose(vecs)
    
end

In [ ]:
corr_matrix = correlation_matrix(20, 10)

# make this into a complete rotation of correlations matrix and see how well it performs.
# make the size dynamic for better efficiency here (how does this translate to GDMRG?!)
bsize = 6
corr_block = corr_matrix[1:bsize, 1:bsize]

vals = eigvals(corr_block)

v_index = indmax(abs.(vals - 0.5))
v = eigvecs(corr_block)[:,v_index]

fullgate = eye(bsize, bsize)
for pivot=bsize-1:-1:1
    theta = atan(v[pivot + 1]/ v[pivot])
    Ugate = eye(bsize, bsize)
    Ugate[pivot:pivot+1, pivot:pivot+1] = [cos(theta) -sin(theta); sin(theta) cos(theta)]
    fullgate = fullgate * Ugate
    v = transpose(v.' * Ugate)
end
fullgate' * corr_block * fullgate
# how to organize and store the gates?
# how to apply the gates the most efficient way?
# how to implement Gutzwiller projection in this approach?